In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
import time
from pyspark.sql import SparkSession

/tmp/ipykernel_57947/3760276594.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
spark = SparkSession.builder \
       .master("local[*]") \
       .appName("test")\
       .config("spark.driver.extraClassPath", "/dados10t/datalake/raw/omop/docker_imagens/postgresql-42.6.0.jar")\
       .config("spark.driver.memory", "80g")\
       .config("spark.executor.memory", "50g")\
       .config("spark.dirver.maxResultSize", "50g")\
       .config('spark.local.dir', '/dados10t/datalake/raw/checkpoint_dir')\
       .getOrCreate()
sc = spark.sparkContext
sc.setCheckpointDir('/dados10t/datalake/raw/checkpoint_dir')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/20 18:04:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/20 18:04:01 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/10/20 18:04:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


#### reading table

In [3]:
df = spark.read.parquet('/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_visit_occurrence_data.parquet/')

In [8]:
df = df\
.withColumn('visit_occurrence_id', F.col('visit_occurrence_id').cast(IntegerType()))\
.withColumn('person_id', F.col('person_id').cast(IntegerType()))

In [ ]:
df.count()

In [6]:
df.printSchema()

root
 |-- visit_occurrence_id: long (nullable = true)
 |-- visit_concept_id: integer (nullable = true)
 |-- visit_start_date: date (nullable = true)
 |-- visit_end_date: date (nullable = true)
 |-- care_site_id: integer (nullable = true)
 |-- visit_type_concept_id: integer (nullable = true)
 |-- person_id: long (nullable = true)



In [10]:
cnae = spark.read.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_care_site.parquet")
table_p_cnae = df.join(cnae, 'care_site_id' ,'left')
table_p_cnae = table_p_cnae.withColumn('valid_cnae',
                                       F.when(F.col('care_site_source_value').isNull() & F.col('care_site_id').isNotNull(), 0)
                                       .otherwise(1))
table_p_cnae = table_p_cnae.withColumn('care_site_id',
                                       F.when(F.col('valid_cnae') == 0, None)\
                                       .otherwise(F.col('care_site_id'))
                                      )
table_p_cnae = table_p_cnae.select(df.columns)

In [9]:
options = {
    "url": "jdbc:postgresql://157.86.211.201:5432/postgres",
    "dbtable": "omop54_v2.visit_occurrence",
    "user": "postgres",
    "password": "omop2023",
    "postgreSQLConf": "constraint.mode=noconstraints"
}

In [10]:
df.write \
    .format("jdbc") \
    .options(**options) \
    .mode("append") \
    .save()

In [11]:
print("finished")

finished
